# Requirements

In [1]:
import itertools
import numpy as np
import random

# Model

The model consists of agents that have a fixed position in space.  They can be in several states, represented by integers.  At each time step, an agent is selected, as well as a random neighbour (von Neuman neighbourhood).  The agent's state is then changed to that of its neighbour.  The simulation ends when all agents are in the same state.

The space can have any dimension, and periodic boundaries are applied.  So in 1D this is a ring, in 2D a torus, etc.

In [2]:
class Population:
    
    def __init__(self, shape, nr_species=2):
        '''Create a new population
        
        Arguments
        ---------
        shape: tuple[int]
            defines the number of dimentions and the size in each dimension
        nr_species: int
            the number of species to be present initially
        '''
        self._space = np.random.choice(nr_species, size=shape)
        self._neighbours = self._compute_neighbours()
        self._reshape = tuple([-1] + [1]*(self._space.ndim - 1))
        self._index = tuple([0]*(self._space.ndim - 1))
        self._nr_steps = None
        
    def _compute_neighbours(self):
        '''Initialize the neighbour deltas for the given number of dimentions
        '''
        neighbours = set()
        for coordinate in itertools.permutations([1] + [0]*(self._space.ndim - 1),
                                                 self._space.ndim):
            neighbours.add(coordinate)
        for coordinate in itertools.permutations([-1] + [0]*(self._space.ndim - 1),
                                                 self._space.ndim):
            neighbours.add(coordinate)
        return list(map(np.array, neighbours))
    
    def _step(self):
        '''Perform an individual step in the simulation
        
        Randomly select an agent by coordinate, randomly select a neighbour delta,
        if the species are different, update the selected agent, and increment the
        number of steps.
        '''
        coordinate = np.empty((self._space.ndim, ), dtype=np.int32)
        for i, dim in enumerate(self._space.shape):
            coordinate[i] = np.random.choice(dim, size=1)
        species = self._space[tuple(coordinate)]
        neighbour = np.mod(coordinate + random.choice(self._neighbours), self._space.shape)
        new_species = self._space[tuple(neighbour)]
        if species != new_species:
            self._space[tuple(coordinate)] = new_species
            self._nr_steps += 1
        
    def run(self):
        '''Run the simulation until all agents have the same state
        '''
        self._nr_steps = 0
        while self.nr_species > 1:
            self._step()
    
    @property
    def nr_species(self):
        '''Return the number of species
        
        Returs
        ------
        int
            number of species
        '''
        species = set(x[self._index] for x in self._space.reshape(self._reshape))
        return len(species)

    @property
    def nr_steps(self):
        '''Return the number of steps
        
        Returns
        -------
        int
            number of steps taken in the simulation
        '''
        return self._nr_steps

Create a new population on a $5 \times 5$ grid with two distinct states.  Run the simulation and return the number of steps.

In [3]:
population = Population((5, 5), 2)
population.run()
population.nr_steps

89

Create a new population on a $5 \times 5 \times 5$ grid with two distinct states.  Run the simulation and return the number of steps.

In [4]:
population = Population((5, 5, 5), 2)
population.run()
population.nr_steps

7182